<a href="https://colab.research.google.com/github/ShashwatVv/DetectingSaracasm-In-NewsHeadlines/blob/main/Detecting_Sarcasm_in_News_headlines.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###  Let's first load the dataset. 

In [8]:
!wget --no-check-certificate \
    https://storage.googleapis.com/laurencemoroney-blog.appspot.com/sarcasm.json \
    -O /tmp/sarcasm.json

--2022-03-28 10:12:08--  https://storage.googleapis.com/laurencemoroney-blog.appspot.com/sarcasm.json
Resolving storage.googleapis.com (storage.googleapis.com)... 173.194.218.128, 108.177.11.128, 74.125.31.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|173.194.218.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5643545 (5.4M) [application/json]
Saving to: ‘/tmp/sarcasm.json’

/tmp/sarcasm.json   100%[===================>]   5.38M  --.-KB/s    in 0.02s   

2022-03-28 10:12:08 (218 MB/s) - ‘/tmp/sarcasm.json’ saved [5643545/5643545]



In [47]:
##  The libraries to be imported
import numpy  as np
import json 
import io
import tensorflow  as  tf
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.text import Tokenizer  as TKN
from tensorflow.keras.preprocessing.sequence import pad_sequences as padseq
print("Imported!!")

Imported!!


In [48]:
## Let's load the json-data

file_sarc = open('/tmp/sarcasm.json', 'r')
data = json.load(file_sarc)
file_sarc.close()

In [49]:
info = data[0].keys()
print("The 3 features associated with the news headlines are:", *info)

The 3 features associated with the news headlines are: article_link headline is_sarcastic


In [50]:
## let's create separate list of sentences, labels and  links/urls 
list_of_labels = list() ##sarcastic or non sarcastic
list_of_links = list() ##url to the article 
list_of_sentences = list() 

for instance in data:
  list_of_labels.append(instance['is_sarcastic'])
  list_of_links.append(instance['article_link'])
  list_of_sentences.append(instance['headline'])

print("Length of each of these lists are:",  len(list_of_labels), len(list_of_links), len(list_of_sentences))

Length of each of these lists are: 26709 26709 26709


In [51]:
## Let's do  some standard preprocessing!!
##create  a  tokenizer instance
tokenizer_instance = TKN(oov_token = '<OOV>')
##let's fit this object on the sentence lists
tokenizer_instance.fit_on_texts(list_of_sentences)
word_index = tokenizer_instance.word_index

In [52]:
word_index
list_of_sentences[:5]

["former versace store clerk sues over secret 'black code' for minority shoppers",
 "the 'roseanne' revival catches up to our thorny political mood, for better and worse",
 "mom starting to fear son's web series closest thing she will have to grandchild",
 'boehner just wants wife to listen, not come up with alternative debt-reduction ideas',
 'j.k. rowling wishes snape happy birthday in the most magical way']

In [53]:
sequences = tokenizer_instance.texts_to_sequences(list_of_sentences)
##this tep was used to ensure tokens represent the words
padded = padseq(sequences, padding='post') ##--> 'post' describes all the sequences will be padded to the longest sequence
print(padded[0],'\n', padded.shape) 

[  308 15115   679  3337  2298    48   382  2576 15116     6  2577  8434
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0] 
 (26709, 40)


In [54]:
##Let's set the parameters
size_vocab = 10000
embedding_dimension = 16
len_max = 32
truncate = 'post'
padding = 'post'
ukn_token = '<OOV>'
size_train = 20000

In [55]:
## Let's split into  train and tests!!
xtrain, ytrain = list_of_sentences[0:size_train], list_of_labels[0:size_train]
xtest,  ytest =  list_of_sentences[size_train:], list_of_labels[size_train:]

In [58]:
## Let's tokenize and pad  our training and test data

tokenizer = TKN(num_words=size_vocab, oov_token=ukn_token)
tokenizer.fit_on_texts(xtrain)
word_index = tokenizer.word_index

train_seq = tokenizer.texts_to_sequences(xtrain)
train_pad = padseq(train_seq, maxlen=len_max, padding=padding, truncating=truncate)

test_seq = tokenizer.texts_to_sequences(xtest)
test_pad = padseq(test_seq, maxlen=len_max, padding='post', truncating='post')